In [1]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer


def findOccurrence(sentence, word): #find the occurrence of a given word by stemming and lemmatizing the word and the given text sequence 
    tokens = word_tokenize(sentence.lower()) #tokenize the text sequence
    stemmer = PorterStemmer() 
    lemmatizer = WordNetLemmatizer()
    stemmed = [stemmer.stem(token) for token in tokens] #stem each token 
    lemmatized = [lemmatizer.lemmatize(token) for token in tokens]   #lemmatie each token
    word = stemmer.stem(word) #stem the word to search for in the text

    if word in tokens or word in stemmed or word in lemmatized: #if the stemmed word is found with any method
        return True
    else:
        return False

In [2]:
from transformers import AutoTokenizer, AutoModelForCausalLM, GenerationConfig, LlamaTokenizer, LlamaForCausalLM
import transformers
import torch

def generate_prompt(instruction: str, input_ctxt: str = None) -> str: # boilerplate provided by alpaca models
    if input_ctxt:
        return f"""Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.
        ### Instruction:
        {instruction}
        ### Input:
        {input_ctxt}
      	### Response:"""
    else:
      return f"""Below is an instruction that describes a task. Write a response that appropriately completes the request.
    	 ### Instruction:
      {instruction}
       ### Response:"""


model = "/bigwork/nhwpshaa/Fine-tuned AlpacaNative/Few Epochs/3 Epochs/checkpoint-760" #load the model
tokenizer = LlamaTokenizer.from_pretrained("/bigwork/nhwpshaa/alpaca-native/") #load the tokenizer
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
    device_map="auto",
)

You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565


Loading checkpoint shards:   0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
import csv


class effect:
    def __init__(self, name):
        self.name = name
        self.recall = False


class causalrelation:
    def __init__(self, cause):
        self.cause = cause
        self.effect= []
        self.effectRecallCount = 0
        self.recall = False

causalrelations = []

with open('Experiments/Section 5.1: Causal Relationships Extraction/Extracted Causal Relationship/firstOrderEffects.csv', 'r') as file: #load the filw with the causal relations
    csvfile = csv.DictReader(file)
    for row in csvfile:
        flag = False
        for i in causalrelations:
            if row['cause'] == i.cause: #collect all causes
                newEffect = effect(row['effect']) #initialize effect
                i.effect.append(newEffect) #add the effect to the cause
                flag = True
        if flag == False: #create new cause if not initilized
            newCR = causalrelation(row['cause']) 
            newEffect = effect(row['effect'])
            newCR.effect.append(newEffect)
            causalrelations.append(newCR)

In [5]:
input_ctxt = "What are the effects of <mask>?" #check for every question type


instruction ="Answer the following question to the best of your knowledge"
k = 0
for i in causalrelations:
    i.effectRecallCount = 0 # reset the count for each new effect
    input_ctxt = "What are the effects of <mask>?"
    input_ctxt = input_ctxt.replace("<mask>", i.cause) #replace the mask with the actual cause
    sequences = pipeline(
        generate_prompt(instruction, input_ctxt),
        max_length=160,
        do_sample=True,
        top_k=50,
        num_return_sequences=4,
        eos_token_id=tokenizer.eos_token_id,
    )
    for singleEffect in i.effect:
        singleEffect.recall = False #reset the flag for each effect
        for seq in sequences:
            if findOccurrence(seq['generated_text'], singleEffect.name):
                i.recall = True #set the flag when the effect is found
                singleEffect.recall = True # set the flag when the effect is found
                i.effectRecallCount += 1 #increment the number of effects found
                break #move on to the next effect

k=0
for relations in causalrelations:
    if relations.recall == True: #calculate the total recalled topics
        k += 1

print(f"Total recalled topics: {k}")
print(f"Total topics: {len(causalrelations)}")
with open("/bigwork/nhwpshaa/Fine-tuned AlpacaNative/Few Epochs/3 Epochs/AlpacaStatsWhat.csv", 'w', newline='') as writefile:
    fieldnames = ['cause', 'total effects', 'recalled effects']
    writer = csv.DictWriter(writefile, fieldnames=fieldnames)
    writer.writeheader()
    for printvalues in causalrelations:
        recallcount= 0
        print(f"Total no. of effects: {len(printvalues.effect)}")
        for oneEffect in printvalues.effect:
            if oneEffect.recall == True: #calculate the total recalled effects
                recallcount += 1
        print(f"Total recalled: {recallcount}")
        writer.writerow({'cause': printvalues.cause, 'total effects': len(printvalues.effect), 'recalled effects': recallcount})
        for values in printvalues.effect:
            print(f"Cause: {printvalues.cause} Effect: {values.name} Recalled(Effect): {values.recall}")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


Total recalled topics: 10
Total topics: 52
Total no. of effects: 104
Total recalled: 5
Cause: crime Effect: death Recalled(Effect): True
Cause: crime Effect: imprisonment Recalled(Effect): False
Cause: crime Effect: crime Recalled(Effect): True
Cause: crime Effect: conviction Recalled(Effect): False
Cause: crime Effect: harm Recalled(Effect): False
Cause: crime Effect: pregnancy Recalled(Effect): False
Cause: crime Effect: death of another person Recalled(Effect): False
Cause: crime Effect: financial loss Recalled(Effect): False
Cause: crime Effect: loss of life Recalled(Effect): False
Cause: crime Effect: great bodily injury Recalled(Effect): False
Cause: crime Effect: costs Recalled(Effect): True
Cause: crime Effect: outrage Recalled(Effect): False
Cause: crime Effect: conflict Recalled(Effect): False
Cause: crime Effect: concern Recalled(Effect): False
Cause: crime Effect: gangs Recalled(Effect): False
Cause: crime Effect: ignite war Recalled(Effect): False
Cause: crime Effect: emot

In [6]:
instruction ="Answer the following question to the best of your knowledge"
k = 0
for i in causalrelations:
    i.effectRecallCount = 0
    input_ctxt = "What impacts does <mask> have?"
    input_ctxt = input_ctxt.replace("<mask>", i.cause)
    sequences = pipeline(
        generate_prompt(instruction, input_ctxt),
        max_length=160,
        do_sample=True,
        top_k=50,
        num_return_sequences=4,
        eos_token_id=tokenizer.eos_token_id,
    )
    for singleEffect in i.effect:
        singleEffect.recall = False
        for seq in sequences:
            if findOccurrence(seq['generated_text'], singleEffect.name):
                i.recall = True
                singleEffect.recall = True
                i.effectRecallCount += 1
                #k += 1
                break

k=0
for relations in causalrelations:
    if relations.recall == True:
        k += 1

print(f"Total recalled topics: {k}")
print(f"Total topics: {len(causalrelations)}")
with open("/bigwork/nhwpshaa/Fine-tuned AlpacaNative/Few Epochs/3 Epochs/AlpacaStatsWhat.csv", 'a', newline='') as writefile:
    fieldnames = ['cause', 'total effects', 'recalled effects']
    writer = csv.DictWriter(writefile, fieldnames=fieldnames)
    writer.writeheader()
    for printvalues in causalrelations:
        recallcount= 0
        print(f"Total no. of effects: {len(printvalues.effect)}")
        for oneEffect in printvalues.effect:
            if oneEffect.recall == True:
                recallcount += 1
        print(f"Total recalled: {recallcount}")
        writer.writerow({'cause': printvalues.cause, 'total effects': len(printvalues.effect), 'recalled effects': recallcount})
        for values in printvalues.effect:
            print(f"Cause: {printvalues.cause} Effect: {values.name} Recalled(Effect): {values.recall}")

Total recalled topics: 13
Total topics: 52
Total no. of effects: 104
Total recalled: 3
Cause: crime Effect: death Recalled(Effect): False
Cause: crime Effect: imprisonment Recalled(Effect): False
Cause: crime Effect: crime Recalled(Effect): True
Cause: crime Effect: conviction Recalled(Effect): False
Cause: crime Effect: harm Recalled(Effect): True
Cause: crime Effect: pregnancy Recalled(Effect): False
Cause: crime Effect: death of another person Recalled(Effect): False
Cause: crime Effect: financial loss Recalled(Effect): False
Cause: crime Effect: loss of life Recalled(Effect): False
Cause: crime Effect: great bodily injury Recalled(Effect): False
Cause: crime Effect: costs Recalled(Effect): True
Cause: crime Effect: outrage Recalled(Effect): False
Cause: crime Effect: conflict Recalled(Effect): False
Cause: crime Effect: concern Recalled(Effect): False
Cause: crime Effect: gangs Recalled(Effect): False
Cause: crime Effect: ignite war Recalled(Effect): False
Cause: crime Effect: emot

In [ ]:
instruction ="Answer the following question to the best of your knowledge"
k = 0
for i in causalrelations:
    i.effectRecallCount = 0
    input_ctxt = "What effect does <mask> have?"
    input_ctxt = input_ctxt.replace("<mask>", i.cause)
    sequences = pipeline(
        generate_prompt(instruction, input_ctxt),
        max_length=160,
        do_sample=True,
        top_k=50,
        num_return_sequences=4,
        eos_token_id=tokenizer.eos_token_id,
    )
    for singleEffect in i.effect:
        singleEffect.recall = False
        for seq in sequences:
            if findOccurrence(seq['generated_text'], singleEffect.name):
                i.recall = True
                singleEffect.recall = True
                i.effectRecallCount += 1
                #k += 1
                break
                
                
k=0
for relations in causalrelations:
    if relations.recall == True:
        k += 1

print(f"Total recalled topics: {k}")
print(f"Total topics: {len(causalrelations)}")
with open("/bigwork/nhwpshaa/Fine-tuned AlpacaNative/Few Epochs/3 Epochs/AlpacaStatsWhat.csv", 'a', newline='') as writefile:
    fieldnames = ['cause', 'total effects', 'recalled effects']
    writer = csv.DictWriter(writefile, fieldnames=fieldnames)
    for printvalues in causalrelations:
        recallcount= 0
        print(f"Total no. of effects: {len(printvalues.effect)}")
        for oneEffect in printvalues.effect:
            if oneEffect.recall == True:
                recallcount += 1
        print(f"Total recalled: {recallcount}")
        writer.writerow({'cause': printvalues.cause, 'total effects': len(printvalues.effect), 'recalled effects': recallcount})
        for values in printvalues.effect:
            print(f"Cause: {printvalues.cause} Effect: {values.name} Recalled(Effect): {values.recall}")

In [8]:
instruction ="Answer the following question to the best of your knowledge"
k = 0
for i in causalrelations:
    i.effectRecallCount = 0
    input_ctxt = "What does <mask> lead to?"
    input_ctxt = input_ctxt.replace("<mask>", i.cause)
    sequences = pipeline(
        generate_prompt(instruction, input_ctxt),
        max_length=160,
        do_sample=True,
        top_k=50,
        num_return_sequences=4,
        eos_token_id=tokenizer.eos_token_id,
    )
    for singleEffect in i.effect:
        singleEffect.recall = False
        for seq in sequences:
            if findOccurrence(seq['generated_text'], singleEffect.name):
                i.recall = True
                singleEffect.recall = True
                i.effectRecallCount += 1
                break

                
k=0
for relations in causalrelations:
    if relations.recall == True:
        k += 1

print(f"Total recalled topics: {k}")
print(f"Total topics: {len(causalrelations)}")
with open("/bigwork/nhwpshaa/Fine-tuned AlpacaNative/Few Epochs/3 Epochs/AlpacaStatsWhat.csv", 'a', newline='') as writefile:
    fieldnames = ['cause', 'total effects', 'recalled effects']
    writer = csv.DictWriter(writefile, fieldnames=fieldnames)
    for printvalues in causalrelations:
        recallcount= 0
        print(f"Total no. of effects: {len(printvalues.effect)}")
        for oneEffect in printvalues.effect:
            if oneEffect.recall == True:
                recallcount += 1
        print(f"Total recalled: {recallcount}")
        writer.writerow({'cause': printvalues.cause, 'total effects': len(printvalues.effect), 'recalled effects': recallcount})
        for values in printvalues.effect:
            print(f"Cause: {printvalues.cause} Effect: {values.name} Recalled(Effect): {values.recall}")

Total recalled topics: 13
Total topics: 52
Total no. of effects: 104
Total recalled: 4
Cause: crime Effect: death Recalled(Effect): True
Cause: crime Effect: imprisonment Recalled(Effect): True
Cause: crime Effect: crime Recalled(Effect): True
Cause: crime Effect: conviction Recalled(Effect): False
Cause: crime Effect: harm Recalled(Effect): False
Cause: crime Effect: pregnancy Recalled(Effect): False
Cause: crime Effect: death of another person Recalled(Effect): False
Cause: crime Effect: financial loss Recalled(Effect): False
Cause: crime Effect: loss of life Recalled(Effect): False
Cause: crime Effect: great bodily injury Recalled(Effect): False
Cause: crime Effect: costs Recalled(Effect): False
Cause: crime Effect: outrage Recalled(Effect): False
Cause: crime Effect: conflict Recalled(Effect): False
Cause: crime Effect: concern Recalled(Effect): False
Cause: crime Effect: gangs Recalled(Effect): False
Cause: crime Effect: ignite war Recalled(Effect): False
Cause: crime Effect: emot

In [ ]:
instruction ="Answer the following question to the best of your knowledge"
k = 0
for i in causalrelations:
    i.effectRecallCount = 0
    for singleEffect in i.effect:
        singleEffect.recall = False
        input_ctxt = "What factors are associated with <mask>?"
        input_ctxt = input_ctxt.replace("<mask>", singleEffect.name)
        sequences = pipeline(
            generate_prompt(instruction, input_ctxt),
            max_length=160,
            do_sample=True,
            top_k=50,
            num_return_sequences=4,
            eos_token_id=tokenizer.eos_token_id,
        )
        for seq in sequences:
            if findOccurrence(seq['generated_text'], i.cause):
                    i.recall = True
                    singleEffect.recall = True
                    i.effectRecallCount += 1
                    #k += 1
                    break
                   
                
k=0
for relations in causalrelations:
    if relations.recall == True:
        k += 1

print(f"Total recalled topics: {k}")
print(f"Total topics: {len(causalrelations)}")
with open("/bigwork/nhwpshaa/Fine-tuned AlpacaNative/Few Epochs/3 Epochs/AlpacaStatsWhat.csv", 'a', newline='') as writefile:
    fieldnames = ['cause', 'total effects', 'recalled effects']
    writer = csv.DictWriter(writefile, fieldnames=fieldnames)
    for printvalues in causalrelations:
        recallcount= 0
        print(f"Total no. of effects: {len(printvalues.effect)}")
        for oneEffect in printvalues.effect:
            if oneEffect.recall == True:
                recallcount += 1
        print(f"Total recalled: {recallcount}")
        writer.writerow({'cause': printvalues.cause, 'total effects': len(printvalues.effect), 'recalled effects': recallcount})
        for values in printvalues.effect:
            print(f"Cause: {printvalues.cause} Effect: {values.name} Recalled(Effect): {values.recall}")

In [ ]:
instruction ="Answer the following question to the best of your knowledge"
k = 0
for i in causalrelations:
    i.effectRecallCount = 0
    input_ctxt = "What happens due to <mask>?"
    input_ctxt = input_ctxt.replace("<mask>", i.cause)
    sequences = pipeline(
        generate_prompt(instruction, input_ctxt),
        max_length=160,
        do_sample=True,
        top_k=50,
        num_return_sequences=4,
        eos_token_id=tokenizer.eos_token_id,
    )
    for singleEffect in i.effect:
        singleEffect.recall = False
        for seq in sequences:
            if findOccurrence(seq['generated_text'], singleEffect.name):
                i.recall = True
                singleEffect.recall = True
                i.effectRecallCount += 1
                break

                
k=0
for relations in causalrelations:
    if relations.recall == True:
        k += 1

print(f"Total recalled topics: {k}")
print(f"Total topics: {len(causalrelations)}")
with open("/bigwork/nhwpshaa/Fine-tuned AlpacaNative/Few Epochs/3 Epochs/AlpacaStatsWhat.csv", 'a', newline='') as writefile:
    fieldnames = ['cause', 'total effects', 'recalled effects']
    writer = csv.DictWriter(writefile, fieldnames=fieldnames)
    for printvalues in causalrelations:
        recallcount= 0
        print(f"Total no. of effects: {len(printvalues.effect)}")
        for oneEffect in printvalues.effect:
            if oneEffect.recall == True:
                recallcount += 1
        print(f"Total recalled: {recallcount}")
        writer.writerow({'cause': printvalues.cause, 'total effects': len(printvalues.effect), 'recalled effects': recallcount})
        for values in printvalues.effect:
            print(f"Cause: {printvalues.cause} Effect: {values.name} Recalled(Effect): {values.recall}")

In [ ]:
##Below this

In [ ]:
instruction ="Answer the following question to the best of your knowledge"
k = 0
for i in causalrelations:
    i.effectRecallCount = 0
    for singleEffect in i.effect:
        singleEffect.recall = False
        input_ctxt = "What causes <mask>?"
        input_ctxt = input_ctxt.replace("<mask>", singleEffect.name)
        sequences = pipeline(
            generate_prompt(instruction, input_ctxt),
            max_length=160,
            do_sample=True,
            top_k=50,
            num_return_sequences=4,
            eos_token_id=tokenizer.eos_token_id,
        )
        for seq in sequences:
            if findOccurrence(seq['generated_text'], i.cause):
                    i.recall = True
                    singleEffect.recall = True
                    i.effectRecallCount += 1
                    break
                  
                
k=0
for relations in causalrelations:
    if relations.recall == True:
        k += 1

print(f"Total recalled topics: {k}")
print(f"Total topics: {len(causalrelations)}")
with open("/bigwork/nhwpshaa/Fine-tuned AlpacaNative/Few Epochs/3 Epochs/AlpacaStatsWhat.csv", 'a', newline='') as writefile:
    fieldnames = ['cause', 'total effects', 'recalled effects']
    writer = csv.DictWriter(writefile, fieldnames=fieldnames)
    for printvalues in causalrelations:
        recallcount= 0
        print(f"Total no. of effects: {len(printvalues.effect)}")
        for oneEffect in printvalues.effect:
            if oneEffect.recall == True:
                recallcount += 1
        print(f"Total recalled: {recallcount}")
        writer.writerow({'cause': printvalues.cause, 'total effects': len(printvalues.effect), 'recalled effects': recallcount})
        for values in printvalues.effect:
            print(f"Cause: {printvalues.cause} Effect: {values.name} Recalled(Effect): {values.recall}")

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


In [ ]:
import random

nouns = [
    "science", "technology", "politics", "geography", "knowledge",
    "chemistry", "biology", "physics", "astronomy", "engineering",
    "computer", "internet", "software", "hardware", "algorithm",
    "government", "democracy", "election", "policy", "diplomacy",
    "continent", "country", "city", "ocean", "mountain",
    "planet", "star", "galaxy", "earthquake", "volcano",
    "invention", "discovery", "innovation", "experiment", "research",
    "energy", "environment", "climate", "weather", "ecosystem",
    "history", "culture", "civilization", "tradition", "heritage",
    "language", "literature", "education", "society", "population",
    "cancer", "Albert Einstein", "DNA", "vaccine", "robotics",
    "nanotechnology", "climate change", "sustainability", "globalization",
    "terrorism", "human rights", "economy", "finance", "trade",
    "migration", "demography", "geology", "meteorology", "biodiversity",
    "psychology", "philosophy", "artificial intelligence", "space exploration",
    "genetics", "biotechnology", "medicine", "healthcare", "nanoscience",
    "particle physics", "quantum mechanics", "cosmology", "astrophysics",
    "fossil fuels", "renewable energy", "deforestation", "urbanization",
    "archaeology", "anthropology", "linguistics", "geopolitics", "cartography",
    "sociology", "ethics", "paleontology", "pandemic", "virus"
]



input_ctxt = f"What is a result of <cause>? A){random.choice(nouns)} B){random.choice(nouns)} C) <effect>" ###redo


instruction ="Answer the following question to the best of your knowledge"
k = 0
count = 0
for i in causalrelations:
    i.effectRecallCount = 0
    for n in i.effect:
        n.recall = False
        input_ctxt = f"What is a result of <cause>? A){random.choice(nouns)} B){random.choice(nouns)} C) <effect>"
        input_ctxt = input_ctxt.replace("<effect>", n.name)
        input_ctxt = input_ctxt.replace("<cause>", i.cause)
        #print(input_ctxt)
        sequences = pipeline(
            generate_prompt(instruction, input_ctxt),
            max_length=160,
            do_sample=True,
            top_k=50,
            num_return_sequences=4,
            eos_token_id=tokenizer.eos_token_id,
        )
        for seq in sequences:
            if findOccurrence(seq['generated_text'], n.name):
                #print(f"#################################{j}##############################")
                #print(f"Result: {seq['generated_text']}")
                i.recall = True
                n.recall = True
                i.effectRecallCount += 1
                break
               
            
k=0
for relations in causalrelations:
    if relations.recall == True:
        k += 1
print(f"Total recalled topics: {k}")
print(f"Total topics: {len(causalrelations)}")
with open("/bigwork/nhwpshaa/Fine-tuned AlpacaNative/Few Epochs/3 Epochs/AlpacaStatsPreCursorMultipleChoice.csv", 'w', newline='') as writefile:
    fieldnames = ['cause', 'total effects', 'recalled effects']
    writer = csv.DictWriter(writefile, fieldnames=fieldnames)
    writer.writeheader()
    for printvalues in causalrelations:
        recallcount= 0
        print(f"Total no. of effects: {len(printvalues.effect)}")
        for oneEffect in printvalues.effect:
            if oneEffect.recall == True:
                recallcount += 1
        print(f"Total recalled: {recallcount}")
        writer.writerow({'cause': printvalues.cause, 'total effects': len(printvalues.effect), 'recalled effects': recallcount})
        for values in printvalues.effect:
            print(f"Cause: {printvalues.cause} Effect: {values.name} Recalled(Effect): {values.recall}")

In [ ]:
input_ctxt = "Can <cause> cause <effect>?"


instruction ="Answer the following question to the best of your knowledge"
k = 0
count = 0
for i in causalrelations:
    i.effectRecallCount = 0
    for singleEffect in i.effect: 
        singleEffect.recall = False
        input_ctxt = "Can <cause> cause <effect>?"
        input_ctxt = input_ctxt.replace("<effect>", singleEffect.name)
        input_ctxt = input_ctxt.replace("<cause>", i.cause)
        sequences = pipeline(
            generate_prompt(instruction, input_ctxt),
            max_length=160,
            do_sample=True,
            top_k=50,
            num_return_sequences=4,
            eos_token_id=tokenizer.eos_token_id,
        )
        for seq in sequences:
            if findOccurrence(seq['generated_text'], "yes"):
                i.recall = True
                i.effectRecallCount += 1
                singleEffect.recall = True
                #k += 1
                #print("We should break out of the loop now!")
                break

            
k=0
for relations in causalrelations:
    if relations.recall == True:
        k += 1
print(f"Total recalled topics: {k}")
print(f"Total topics: {len(causalrelations)}")
with open("/bigwork/nhwpshaa/Fine-tuned AlpacaNative/Few Epochs/3 Epochs/AlpacaStatsYesNo.csv", 'w', newline='') as writefile:
    fieldnames = ['cause', 'total effects', 'recalled effects']
    writer = csv.DictWriter(writefile, fieldnames=fieldnames)
    writer.writeheader()
    for printvalues in causalrelations:
        recallcount= 0
        print(f"Total no. of effects: {len(printvalues.effect)}")
        for oneEffect in printvalues.effect:
            if oneEffect.recall == True:
                recallcount += 1
        print(f"Total recalled: {recallcount}")
        writer.writerow({'cause': printvalues.cause, 'total effects': len(printvalues.effect), 'recalled effects': recallcount})
        for values in printvalues.effect:
            print(f"Cause: {printvalues.cause} Effect: {values.name} Recalled(Effect): {values.recall}")

In [ ]:
instruction ="Answer the following question to the best of your knowledge"
k = 0
count = 0
for i in causalrelations:
    i.effectRecallCount = 0
    for singleEffect in i.effect: 
        singleEffect.recall = False
        input_ctxt = "Does <cause> contribute to <effect>?"
        input_ctxt = input_ctxt.replace("<effect>", singleEffect.name)
        input_ctxt = input_ctxt.replace("<cause>", i.cause)
        sequences = pipeline(
            generate_prompt(instruction, input_ctxt),
            max_length=160,
            do_sample=True,
            top_k=4,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
        )
        for seq in sequences:
            if findOccurrence(seq['generated_text'], "yes"):
                i.recall = True
                singleEffect.recall = True
                i.effectRecallCount += 1
                break
                
k=0
for relations in causalrelations:
    if relations.recall == True:
        k += 1

                
print(f"Total recalled topics: {k}")
print(f"Total topics: {len(causalrelations)}")
with open("/bigwork/nhwpshaa/Fine-tuned AlpacaNative/Few Epochs/3 Epochs/AlpacaStatsYesNo.csv", 'a', newline='') as writefile:
    fieldnames = ['cause', 'total effects', 'recalled effects']
    writer = csv.DictWriter(writefile, fieldnames=fieldnames)
    for printvalues in causalrelations:
        recallcount= 0
        print(f"Total no. of effects: {len(printvalues.effect)}")
        for oneEffect in printvalues.effect:
            if oneEffect.recall == True:
                recallcount += 1
        print(f"Total recalled: {recallcount}")
        writer.writerow({'cause': printvalues.cause, 'total effects': len(printvalues.effect), 'recalled effects': recallcount})
        for values in printvalues.effect:
            print(f"Cause: {printvalues.cause} Effect: {values.name} Recalled(Effect): {values.recall}")

In [ ]:
input_ctxt = "Does <cause> lead to <effect>?"


instruction ="Answer the following question to the best of your knowledge"
k = 0
count = 0
for i in causalrelations:
    i.effectRecallCount = 0
    for singleEffect in i.effect: 
        singleEffect.recall = False
        input_ctxt = "Does <cause> lead to <effect>?"
        input_ctxt = input_ctxt.replace("<effect>", singleEffect.name)
        input_ctxt = input_ctxt.replace("<cause>", i.cause)
        #print(input_ctxt)
        sequences = pipeline(
            generate_prompt(instruction, input_ctxt),
            max_length=160,
            do_sample=True,
            top_k=4,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
        )
        for seq in sequences:
            if findOccurrence(seq['generated_text'], "yes"):
                i.recall = True
                singleEffect.recall = True
                i.effectRecallCount += 1
                break

k=0
for relations in causalrelations:
    if relations.recall == True:
        k += 1


print(f"Total recalled topics: {k}")
print(f"Total topics: {len(causalrelations)}")
with open("/bigwork/nhwpshaa/Fine-tuned AlpacaNative/Few Epochs/3 Epochs/AlpacaStatsYesNo.csv", 'a', newline='') as writefile:
    fieldnames = ['cause', 'total effects', 'recalled effects']
    writer = csv.DictWriter(writefile, fieldnames=fieldnames)
    for printvalues in causalrelations:
        recallcount= 0
        print(f"Total no. of effects: {len(printvalues.effect)}")
        for oneEffect in printvalues.effect:
            if oneEffect.recall == True:
                recallcount += 1
        print(f"Total recalled: {recallcount}")
        writer.writerow({'cause': printvalues.cause, 'total effects': len(printvalues.effect), 'recalled effects': recallcount})
        for values in printvalues.effect:
            print(f"Cause: {printvalues.cause} Effect: {values.name} Recalled(Effect): {values.recall}")

In [ ]:
instruction ="Answer the following question to the best of your knowledge"
k = 0
count = 0
for i in causalrelations:
    i.effectRecallCount = 0
    for singleEffect in i.effect:
        singleEffect.recall = False
        input_ctxt = "How is <effect> caused?"
        input_ctxt = input_ctxt.replace("<effect>", singleEffect.name)
        input_ctxt = input_ctxt.replace("<cause>", i.cause)
        #print(input_ctxt)
        sequences = pipeline(
            generate_prompt(instruction, input_ctxt),
            max_length=160,
            do_sample=True,
            top_k=4,
            num_return_sequences=1,
            eos_token_id=tokenizer.eos_token_id,
        )
        for seq in sequences:
            if findOccurrence(seq['generated_text'], i.cause):
                i.recall = True
                singleEffect.recall = True
                i.effectRecallCount += 1
                break
                

k=0
for relations in causalrelations:
    if relations.recall == True:
        k += 1
                
print(f"Total recalled topics: {k}")
print(f"Total topics: {len(causalrelations)}")
with open("/bigwork/nhwpshaa/Fine-tuned AlpacaNative/Few Epochs/3 Epochs/AlpacaStatsHow.csv", 'w', newline='') as writefile:
    fieldnames = ['cause', 'total effects', 'recalled effects']
    writer = csv.DictWriter(writefile, fieldnames=fieldnames)
    writer.writeheader()
    for printvalues in causalrelations:
        recallcount= 0
        print(f"Total no. of effects: {len(printvalues.effect)}")
        for oneEffect in printvalues.effect:
            if oneEffect.recall == True:
                recallcount += 1
        print(f"Total recalled: {recallcount}")
        writer.writerow({'cause': printvalues.cause, 'total effects': len(printvalues.effect), 'recalled effects': recallcount})
        for values in printvalues.effect:
            print(f"Cause: {printvalues.cause} Effect: {values.name} Recalled(Effect): {values.recall}")